In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import library

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.image import ImageDataGenerator 
from zipfile import ZipFile

In [ ]:
ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r").extractall()
ZipFile("/kaggle/input/dogs-vs-cats/test1.zip", "r").extractall()

In [ ]:
train_path = "./train"
test_path = "./test1"

In [ ]:
# Get images of dogs and cats
file_names = os.listdir(train_path)
categories = []
for path in file_names:
    animal_type = path.split('.')[0]
    categories.append(animal_type)

In [ ]:
df = pd.DataFrame({'Image': file_names, 'Animal': categories})
df.sample(5)

In [ ]:
df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42)

df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)

print(df_train.shape)
print(df_valid.shape)

In [ ]:
# Augment training data
datagen_train = ImageDataGenerator(rescale=1./255, rotation_range=30,
                              width_shift_range=0.1, height_shift_range=0.1,
                              shear_range=0.2, zoom_range=0.2,
                              horizontal_flip=True, fill_mode='nearest')

# Augment validating data
datagen_valid = ImageDataGenerator(rescale=1./255)

In [ ]:
aug_train = datagen_train.flow_from_dataframe(df_train, directory=train_path, 
                                             x_col='Image', y_col='Animal',
                                             target_size=(224, 224), class_mode='binary',
                                             batch_size=64)

aug_valid = datagen_valid.flow_from_dataframe(df_valid, directory=train_path,
                                             x_col='Image', y_col='Animal',
                                             target_size=(224, 224), class_mode='binary',
                                             batch_size=64)

In [ ]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1,
                           save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=9, verbose=1)
rdlr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=4, min_lr=0.00001, verbose=1)

callbacks = [checkpoint, rdlr, es]

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform',
                padding='same', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))  

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.35))
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(2, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

In [ ]:
model.summary()

In [ ]:
hist = model.fit(aug_train, validation_data=aug_valid, epochs=50, verbose=1, callbacks=callbacks)

In [ ]:
plt.figure(figsize=(18, 4))

plt.subplot(121)
plt.title('Loss')
plt.plot(hist.history['loss'], label='training loss')
plt.plot(hist.history['val_loss'], label='validation loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(122)
plt.title('Accuracy')
plt.plot(hist.history['accuracy'], label='training accuracy')
plt.plot(hist.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.savefig('Learning rate.png')
plt.show()

In [ ]:
best_model = load_model('best_model.h5')

In [ ]:
file_names = os.listdir(test_path)
df_test = pd.DataFrame({'Image':file_names})

df_test.sample(5)

In [ ]:
aug_test = datagen_valid.flow_from_dataframe(df_test, directory=test_path, x_col='Image',
                                             y_col=None, class_mode=None, target_size= (224, 224),
                                             batch_size=64, shuffle=False)

In [ ]:
pred = best_model.predict(aug_test)

In [ ]:
pred = np.where(pred > 0.5, 'Dog', 'Cat')

In [ ]:
plt.figure(figsize=(15,15))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(aug_test[0][i])
    plt.xlabel(pred[i])
    
plt.show()

In [ ]:
label = np.where(pred == 'Cat', 0, 1)

In [ ]:
df_submission = pd.read_csv('../input/dogs-vs-cats/sampleSubmission.csv')
df_submission['label'] = label
df_submission.to_csv('submission.csv', index=False)